In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(f"gs://marketing_content_lbg/marketing_data_v5a.csv", encoding="latin")
df.head()

In [ ]:
def change_size_label(s):
    if s == "XS":
        return  "less than 10 words"
    if s == "S":
        return "between 10 and 20 words"
    if s == "M":
        return "between 20 and 40 words"
    if s == "L":
        return "between 40 and 60 words"
    if s == "XL":
        return "more than 60 words"

In [ ]:
df["size_label"] = df["size_label"].apply(change_size_label)

In [ ]:
df

In [ ]:
def headline_prompt(topic, aim):
    return (
        "You work for a bank creating marketing content. "
        "The tone of voice must be quietly confident, expert "
        "and empathetic. Using the following features to guide you, "
        "create a headline between 3 and 7 words for the business banking ad campaign. "
        f"Topic: '{topic}', "
        f"aim: '{aim}'"
        "The ad campaign headline is:"
    )
df["headline_prompt"] = df.apply(
    lambda x: headline_prompt(
        x["ad"],
        x["prompt"],
    ), axis=1
)

def main_content_prompt(headline, size_label, topic, aim):
    return (
        "You work for a bank creating marketing content. "
        "The tone of voice must be quietly confident, expert "
        "and empathetic. Using the following features to guide you, "
        "create the main content for the business banking ad "
        f"Headline: '{headline}', "
        f"campaign {size_label}. "
        f"Topic: '{topic}', "
        f"aim: '{aim}'"
        f"The ad campaign main content {size_label} is:"
    )
df["main_content_prompt"] = df.apply(
    lambda x: main_content_prompt(
        x["Headline"],
        x["size_label"],
        x["ad"],
        x["prompt"],
    ), axis=1
)

def cta_prompt(main_content):
    return (
        "You work for a bank creating marketing content. "
        "The tone of voice must be quietly confident, expert "
        "and empathetic. Using the following ad content, "
        "create a call to action. E.g. "
        "CONTENT: 'You can make a payment to an existing payee in our app'"
        "CALL TO ACTION: 'Download the app'"
        f"CONTENT: {main_content}"
        "CALL TO ACTION:"
    )
df["cta_prompt"] = df.apply(
    lambda x: cta_prompt(
        x["main_content_prompt"]
    ), axis=1
)

In [ ]:
df.head(1)

In [ ]:
df_ft = pd.DataFrame(
    {
        "prompts": pd.concat([
            df["headline_prompt"],
            df["main_content_prompt"],
            df["cta_prompt"],
        ]),
        "content": pd.concat([
            df["Headline"],
            df["description"],
            df["CTA"],
        ]),
    }
)
df_ft = df_ft[df_ft["content"].notna()]

In [ ]:
for i in df_ft["prompts"]:
    print(i)
    break

In [ ]:
df_ft = df_ft.reset_index().drop("index", axis=1)

In [ ]:
df_ft.head()

In [ ]:
df_ft_json = []
for i, df_ in df_ft.iterrows():
    df_ft_json.append({
        "input_text": df_["prompts"], "output_text": df_["content"]
    })


with open("marketing_content.jsonl","w", encoding='utf-8') as jsonfile:
    for item in df_ft_json:
        jsonfile.write(json.dumps(item) + "\n")

In [ ]:
pd.read_json("gs://marketing_content_lbg/marketing_content.jsonl", lines=True, encoding="utf-8")